## Get Data

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import fitbit
import sys
sys.path.append('..')

# gather_keys_oauth2.py file needs to be in the same directory. 
# also needs to install cherrypy: https://pypi.org/project/CherryPy/
# pip install CherryPy
import src.APIs.fitbitLocalAuthorization as Oauth2
import pandas as pd 
import datetime
from src import getSecret

server=Oauth2.OAuth2Server()
server.browser_authorize()
CLIENT_ID=getSecret('FITBIT_CLIENT_ID')
CLIENT_SECRET = getSecret('FITBIT_CLIENT_SECRET')

ACCESS_TOKEN=str(server.fitbit.client.session.token['access_token'])
REFRESH_TOKEN=str(server.fitbit.client.session.token['refresh_token'])
auth2_client=fitbit.Fitbit(CLIENT_ID,CLIENT_SECRET,oauth2=True,access_token=ACCESS_TOKEN,refresh_token=REFRESH_TOKEN)

[11/Apr/2023:16:25:54] ENGINE Listening for SIGTERM.
[11/Apr/2023:16:25:54] ENGINE Listening for SIGHUP.
[11/Apr/2023:16:25:54] ENGINE Listening for SIGUSR1.
[11/Apr/2023:16:25:54] ENGINE Bus STARTING
CherryPy Checker:
The Application mounted at '' has an empty config.

[11/Apr/2023:16:25:54] ENGINE Started monitor thread 'Autoreloader'.
[11/Apr/2023:16:25:54] ENGINE Serving on http://127.0.0.1:8080
[11/Apr/2023:16:25:54] ENGINE Bus STARTED


Opening browser to: https://www.fitbit.com/oauth2/authorize?response_type=code&client_id=23QVYP&redirect_uri=http%3A%2F%2F127.0.0.1%3A8080%2Fauthorize&scope=activity+nutrition+heartrate+location+nutrition+profile+settings+sleep+social+weight&state=L1dyxU6WHRkukpc9keNfzECH11rMvA
127.0.0.1 - - [11/Apr/2023:16:25:56] "GET /authorize?code=159d0e8c33cdf40835413eadebe18c6476068ac9&state=L1dyxU6WHRkukpc9keNfzECH11rMvA HTTP/1.1" 200 122 "" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.0 Safari/605.1.15"


[11/Apr/2023:16:25:57] ENGINE Bus STOPPING
[11/Apr/2023:16:25:57] ENGINE HTTP Server cherrypy._cpwsgi_server.CPWSGIServer(('127.0.0.1', 8080)) shut down
[11/Apr/2023:16:25:57] ENGINE Stopped thread 'Autoreloader'.
[11/Apr/2023:16:25:57] ENGINE Bus STOPPED
[11/Apr/2023:16:25:57] ENGINE Bus EXITING
[11/Apr/2023:16:25:57] ENGINE Bus EXITED
[11/Apr/2023:16:25:57] ENGINE Waiting for child threads to terminate...


In [3]:
from datetime import datetime, timedelta
startTime = datetime.strptime('2022-07-03', '%Y-%m-%d')
endTime = datetime.strptime('2022-10-31', '%Y-%m-%d')
endTime = datetime.now().date().strftime("%Y-%m-%d")
startTime = (datetime.now() - timedelta(days=30)).date().strftime("%Y-%m-%d")
date_list = []
df_list = []
allDates = pd.date_range(start=startTime, end = endTime)
for oneDate in allDates:
    oneDate = oneDate.date().strftime("%Y-%m-%d")
    oneDayData = auth2_client.intraday_time_series('activities/heart', base_date=oneDate, detail_level='1sec')
    df = pd.DataFrame(oneDayData['activities-heart-intraday']['dataset'])
    date_list.append(oneDate)
    df_list.append(df)
    
final_df_list = []
for date, df in zip(date_list, df_list):
    if len(df) == 0:
        continue
    df.loc[:, 'date'] = pd.to_datetime(date)
    final_df_list.append(df)

df_heartRate = pd.concat(final_df_list, axis = 0)
df_heartRate['datetime'] = pd.to_datetime(df_heartRate['date']) +pd.to_timedelta(df_heartRate['time'])
display(df_heartRate)

,time,value,date,datetime
0,19:50:52,62,2023-04-02,2023-04-02 19:50:52
1,19:50:57,63,2023-04-02,2023-04-02 19:50:57
2,19:51:02,65,2023-04-02,2023-04-02 19:51:02
3,19:51:07,66,2023-04-02,2023-04-02 19:51:07
4,19:51:12,65,2023-04-02,2023-04-02 19:51:12
...,...,...,...,...
6301,16:23:06,76,2023-04-11,2023-04-11 16:23:06
6302,16:23:16,74,2023-04-11,2023-04-11 16:23:16
6303,16:23:21,73,2023-04-11,2023-04-11 16:23:21
6304,16:23:36,72,2023-04-11,2023-04-11 16:23:36


In [4]:
# save dataframe as pickle file 
df_heartRate.to_pickle('df_heartRate_2023_04_02_to_2023_04_11.pkl')

In [5]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=df_heartRate['datetime'], y=df_heartRate['value'], mode='lines', name='Heart Rate'))

## 8.) Get Sleep Data

In [ ]:
startTime = pd.datetime(year = 2020, month = 1, day = 5)
endTime = pd.datetime.today().date() - datetime.timedelta(days=1)

In [ ]:
allDates = pd.date_range(start=startTime, end = endTime)

In [ ]:
date_list = []
df_list = []
stages_df_list = []

allDates = pd.date_range(start=startTime, end = endTime)

for oneDate in allDates:
    
    oneDate = oneDate.date().strftime("%Y-%m-%d")
    
    oneDayData = auth2_client.sleep(date=oneDate)
    
    # get number of minutes for each stage of sleep and such. 
    stages_df = pd.DataFrame(oneDayData['summary'])

    df = pd.DataFrame(oneDayData['sleep'][0]['minuteData'])
    
    date_list.append(oneDate)
    
    df_list.append(df)
    
    stages_df_list.append(stages_df)
    
final_df_list = []

final_stages_df_list = []

for date, df, stages_df in zip(date_list, df_list, stages_df_list):

    if len(df) == 0:
        continue
    
    df.loc[:, 'date'] = pd.to_datetime(date)
    
    stages_df.loc[:, 'date'] = pd.to_datetime(date)
    
    final_df_list.append(df)
    final_stages_df_list.append(stages_df)

final_df = pd.concat(final_df_list, axis = 0)

final_stages_df = pd.concat(final_stages_df_list, axis = 0)

In [ ]:
columns = final_stages_df.columns[~final_stages_df.columns.isin(['date'])].values

In [ ]:
columns

In [ ]:
pd.concat([final_stages_df[columns] + 2, final_stages_df[['date']]], axis = 1)

In [ ]:

# Export file to csv
final_df.to_csv('minuteSleep' + '.csv', index = False)
final_stages_df.to_csv('minutesStagesSleep' + '.csv', index = True)